In [7]:
import pandas as pd
from dash import Input,Output,Dash,html,dcc,callback
import plotly.express as px 
import plotly.graph_objects as go

css = "https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css"
app = Dash(name="Dashboard 1",external_stylesheets=css)

df_population = pd.read_csv("Indian_Population.csv")
df_aq = pd.read_csv("Air-Quality-Maps-2023-Averaged.csv")
df_power = pd.read_csv("state-UT-wise-power-consumption.csv")

population_df= pd.DataFrame()
population_df[['State/UT','Estimated Population','Rural Population (%)','Urban Population (%)']] = df_population[['State/UT','Estimated Population (2021)','Rural Population (%)','Urban Population (%)']]
power_df = pd.DataFrame()
power_df[['State','GeoCap_MW','AllocCap_MW','SolarCap_MW','Gen_MU','PeakDem_MW','PeakDemDate','ElecSales_MU','ATCLoss_%','Gap_RsPerKWh']] = df_power[['State/UT','Installed Capacity*(Geographical location based) (in MW)','Installed Capacity*(Including Allocated Shares in Joint & Central Sector Utilities) (in MW)','Rooftop Solar Capacity (in MW)', 'Generation (in MU)','Peak Demand (in MW)', 'Peak Demand Date', 'Electricity Sales (in MU)','AT&C Losses (in %)', 'ACS-ARR (Electricity Sales) Gap (in Rs./kWh)']]
aq_df= df_aq[['States/Ut','NO2 (in μg/m³)','SO2 (in μg/m³)','PM10 (in μg/m³)','PM2.5 (in μg/m³)']]

def Populaton_chart(variable='Estimated Population'):
    if variable=='Urban & Rural Population (%)':
        sorted_pop = population_df.sort_values(by='Rural Population (%)',ascending=False)
        fig = px.bar(sorted_pop,x='State/UT',y=['Rural Population (%)','Urban Population (%)'],color='State/UT',title="{}".format(variable),text_auto=True)
        fig.update_layout(paper_bgcolor="#e5ecf6", height=600)
        return fig
    else :
        sorted_pop = population_df.sort_values(by='Estimated Population',ascending=False)
        fig = px.bar(sorted_pop,x='State/UT',y='Estimated Population',color='State/UT',title="{}".format(variable),text_auto=True)
        fig.update_layout(paper_bgcolor="#e5ecf6", height=600)
        return fig

def air_chart(variable='NO2 (in μg/m³)'):
    sort_aq = aq_df.sort_values(by=variable,ascending=False)
    fig = px.bar(sort_aq,x='States/Ut',y=variable,color='States/Ut',title="{}".format(variable),text_auto=True)
    fig.update_layout(paper_bgcolor='#e5ecf6',height=600)
    return fig

def power_chart(variable='PeakDem_MW'):
    sort_power = power_df.sort_values(by=variable,ascending=False)
    fig = px.bar(sort_power,x="State",y=variable,title="{} chart".format(variable),color='State'.format(variable),text_auto=True)
    fig.update_layout(paper_bgcolor='#e5ecf6',height=600)
    return fig

def datasets_show(variable='Energy'):
    df = aq_df if variable == 'Air Quality' else (population_df if variable == 'Population' else power_df)
    fig = go.Figure(
        data=[go.Table(
            header=dict(values=df.columns, align='left'),
            cells=dict(values=[df[col][:100] for col in df.columns], align='left')  # Limit to 100 rows
        )]
    )
    return fig

pop_s=['Estimated Population','Urban & Rural Population (%)']
p_s=['GeoCap_MW','AllocCap_MW','SolarCap_MW','Gen_MU','PeakDem_MW','PeakDemDate','ElecSales_MU', 'ATCLoss_%','Gap_RsPerKWh']
air_s=['NO2 (in μg/m³)','SO2 (in μg/m³)','PM10 (in μg/m³)','PM2.5 (in μg/m³)']
ds_s=['Energy','Air Quality','Population']

pop = dcc.Dropdown(id='pop_dd',options=pop_s,
                   value='Estimated Population',clearable=False)

aq = dcc.Dropdown(id='aq_dd',options=air_s,
                  value='NO2 (in μg/m³)',clearable=False)

power = dcc.Dropdown(id='power_dd',options=p_s,
                     value='PeakDem_MW',clearable=False)

datasets = dcc.Dropdown(id='ds_dd',options=ds_s,
                        value='Energy',clearable=False)

app.layout = html.Div([
    html.Div([
        html.H1("India 2023 Data Analysis",className='text-center fw-bold m-2'),
        html.Br(),
        dcc.Tabs([
            
            dcc.Tab([html.Br(),datasets,html.Br(),                     
                     dcc.Graph(id='dataset',figure=datasets_show())],label="DataSets"),
            dcc.Tab([html.Br(),pop,html.Br(),
                     dcc.Graph(id='population',figure=Populaton_chart())],label="Population"),
            dcc.Tab([html.Br(),aq,html.Br(),
                     dcc.Graph(id='air',figure=air_chart())],label="Air Quality"),
            dcc.Tab([html.Br(),power,html.Br(),
                     dcc.Graph(id='pwr',figure=power_chart())],label="Energy")
        ])
    ],className="col-8 mx-auto")
], style={"background-color": "#e5ecf6", "height": "100vh"})

@callback(Output("dataset","figure"),[Input('ds_dd','value')])
def update_ds(variable):
    return datasets_show(variable)

@callback(Output("population",'figure'),[Input('pop_dd','value')])
def update_pop(variable):
    return Populaton_chart(variable)

@callback(Output("air",'figure'),[Input('aq_dd','value')])
def update_air(variable):
    return air_chart(variable)

@callback(Output("pwr",'figure'),[Input("power_dd","value")])
def update_pwd(variable):
    return power_chart(variable)

if __name__ == "__main__":
    app.run(debug=True)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: can only concatenate str (not "list") to str



---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: can only concatenate str (not "list") to str

